In [ ]:
!pip install numpy scipy gensim googletrans fitz pypdf pandas pyldavis nltk spacy

In [ ]:
!pip install --upgrade googletrans

In [ ]:
import os
from pypdf import PdfReader
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
import random
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.utils import get_random_state

#setting random seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Attempt to import Googletrans; if unavailable, skip translation
try:
    from googletrans import Translator
    translator = Translator()
    do_translate = True
    print("googletrans imported; will translate non-English texts.")
except ImportError:
    print("googletrans not installed; skipping translation step.")
    translator = None
    do_translate = False

# 1. Extract text from all PDFs under 'downloads'
root_dir = '/kaggle/input/strategies-new/downloads'

In [ ]:
documents = []

for dirpath, _, filenames in os.walk(root_dir):
    for fname in sorted(filenames): #sorting to ensure same order each time
        if fname.lower().endswith('.pdf'):
            full_path = os.path.join(dirpath, fname)
            # Extract country from directory structure
            rel_path = os.path.relpath(full_path, root_dir)
            parts = rel_path.split(os.sep)
            country = parts[0] if len(parts) > 1 else 'Unknown'
            #file_paths.append(full_path)
            print(country, fname)
            try:
                doc = PdfReader(full_path)
                print(len(doc.pages))
                text = ""
                for page in doc.pages:
                    try:
                        text += page.extract_text()
                    except:
                        print(fname + "failed to fully extract")
                documents.append({
                            'country': country,
                            'filename': fname,
                            'text': text
                        })
                doc.close()
            except:
                print("Could not read" + fname)

In [ ]:
df_documents = pd.DataFrame(documents)
df_documents.tail()

In [ ]:
df_documents = df_documents.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
df_documents.to_excel("Untranslated_docs1.xlsx", index=False,header=True)
try:
    df_documents.to_csv("Untranslated_docs1.csv", index=False,header=True)
except:
    pass

In [ ]:
#df_documents = pd.read_excel("/kaggle/input/untranslated-docs/Untranslated_docs1.xlsx")
#df_documents.head()

In [ ]:
import textwrap
import asyncio

async def translate_long_text(text, dest="en", chunk_size=4000):
    """
    Splits `text` into chunks of at most `chunk_size` characters,
    translates each chunk, and returns the rejoined translated text.
    """
    # textwrap.wrap will split on whitespace boundaries
    chunks = textwrap.wrap(text, chunk_size)
    translated_chunks = []
    for chunk in chunks:
        # translate() both detects and translates
        async with Translator() as translator:
            translated = await translator.translate(chunk, dest=dest)
            if translated.src == "en":
                return text # return original text if english
            translated_chunks.append(translated.text)
    return " ".join(translated_chunks)

# 2. Translate to English if possible
translated_documents = []
for doc in documents:
    if do_translate:
        print(doc['filename'])
        translation = await translate_long_text(doc['text'])
        doc['text'] = translation
        print('done')
        #except:
            #print("Translation failed for " + doc['country'] + " , " + doc['filename'])
        # otherwise, leave entry['text'] as-is
        translated_documents.append(doc)

In [ ]:
df_translated_documents = pd.DataFrame(translated_documents)
df_translated_documents = df_translated_documents.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
df_translated_documents.to_excel("translated_docs3.xlsx", index=False,header=True)
try:
    df_translated_documents.to_csv("translated_docs3.csv", index=False,header=True)
except:
    pass
df_translated_documents

In [ ]:
italian = df_translated_documents[df_translated_documents["country"] == 'Italy']
italian

In [ ]:
!pip install unidecode

In [ ]:
#preparing for LDA

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from unidecode import unidecode
from nltk.corpus import words

english_vocab = set(words.words())

def remove_unicode(text: str) -> str:
    """
    Transliterate *all* Unicode to ASCII (e.g. ń → n, 𝐢 → i, ü → u),
    then remove any leftover non‐printable characters and collapse whitespace.
    """
    # 1. Unidecode: full transliteration
    text_ascii = unidecode(text)
    # 2. Remove any remaining non-ASCII (should be rare now)
    text_ascii = text_ascii.encode("ascii", "ignore").decode("ascii")
    # 3. Strip control/non-printable characters
    text_ascii = re.sub(r"[^\x20-\x7E]", " ", text_ascii)
    # 4. Collapse whitespace
    text_ascii = re.sub(r"\s+", " ", text_ascii).strip()
    return text_ascii

def remove_country_name(text: str, country: str) -> str:
    """
    Remove occurrences of the country name (word boundary) from the text, case-insensitive.
    """
    pattern = r'\b{}\b'.format(re.escape(country))
    cleaned = re.sub(pattern, '', text, flags=re.IGNORECASE)
    # Collapse whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def fully_clean(text: str) -> str:
    # 1. Decode any literal Python/JSON-style escapes (\n, \u2019, etc.)
    try:
        text = text.encode('utf-8').decode('unicode_escape')
    except Exception:
        # if it fails (e.g. because it's already real newlines), just carry on
        pass

    # 2. Now replace real newlines/tabs with spaces
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    # 3. Transliterate everything to ASCII (ń→n, 𝐢→i, ’→')
    text = unidecode(text)

    # 4. Drop any remaining non-printable or non-ASCII (just in case)
    text = re.sub(r'[^\x20-\x7E]', ' ', text)

    # 5. Collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_text(text):
    #text = text.replace('\r\n', '')
    text = text.replace('\n', '')  # Newline
    text = text.replace('\r', '')  # Carriage return
    text = text.replace('\t', '')  # Tab
    text = text.replace('\b', '')  # Backspace
    text = text.replace('\f', '')  # Form feed
    text = text.replace('\a', '')  # Alert sound
    text = text.replace('\\', '')  # Literal backslash
    #text = text.replace('\\n', '')  # Newline
    #text = text.replace('\\r', '')  # Carriage return
    #text = text.replace('\\t', '')  # Tab
    #text = text.replace('\\b', '')  # Backspace
    #text = text.replace('\\f', '')  # Form feed
    #text = text.replace('\\a', '')  # Alert sound
    #text = text.replace('\\\\', '')  # Literal backslash
    #text = re.sub(r'\S+@\S+', '', text)
    #text = re.sub(r'http\S+', '', text)
    #text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip().lower()

stop_words = set(stopwords.words('english'))
def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return tokens

nlp = spacy.load('en_core_web_sm')
def lemmatize(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB']]

from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy

# 1. Ensure stopwords are available
nltk.download('stopwords')

# 2. Load spaCy model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# 3. Prepare stopword set
stop_words = set(stopwords.words('english'))


In [ ]:
#df_translated_documents['text'] = clean_text(df_translated_documents['text'])
#df_translated_documents['text'] = preprocess(df_translated_documents['text'])
#df_translated_documents['text'] = lemmatize(df_translated_documents['text'])
def preprocess_text(text):
    # Tokenize, lowercase, remove punctuation
    text = clean_text(text)
    tokens = simple_preprocess(text, deacc=True)
    # Remove stopwords
    tokens_nostop = [tok for tok in tokens if tok not in stop_words and len(tok) > 3]
    # Lemmatize with spaCy
    doc = nlp(" ".join(tokens_nostop))
    
    lemmas = [
        token.lemma_ for token in doc
        if token.lemma_.isalpha() and token.lemma_ not in stop_words
    ]
    return lemmas

df_translated_documents['tokens'] = df_translated_documents['text'].apply(fully_clean)
df_translated_documents['tokens'] = df_translated_documents.apply(lambda row: remove_country_name(row['tokens'], row['country']), axis=1)
df_translated_documents['tokens'] = df_translated_documents['tokens'].apply(preprocess_text)
df_translated_documents['tokens'] = df_translated_documents['tokens'].apply(lambda toks: [t for t in toks if t in english_vocab]) # remove remaining non-english / random characters
df_translated_documents.head()



In [ ]:
df_translated_documents.to_excel('final_docs_english.xlsx',index=False)
try:
    df_translated_documents.to_csv('final_docs_english.csv',index=False)
except:
    pass

In [ ]:
import gensim
# 4. Build Gensim dictionary and corpus
dictionary = gensim.corpora.Dictionary(df_translated_documents['tokens'])
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(tokens) for tokens in df_translated_documents['tokens']]

# 5. Split corpus for held-out perplexity
train_corpus, heldout_corpus = train_test_split(corpus, test_size=0.05, random_state=42)

# 6. Grid search over LdaModel hyperparameters with coherence & perplexity
results = []
best_coherence = -np.inf
best_model = None
best_params = {}
alpha='auto'

for num_topics in [5, 10, 15, 20, 25]:
    for passes in [10, 20, 30, 35, 40]:
        for iterations in [25,50,100]:
            print("Training model for ",num_topics," topics")
            model = LdaModel(
                corpus=train_corpus,
                id2word=dictionary,
                num_topics=num_topics,
                alpha='auto',
                eta='auto',
                passes=passes,
                iterations=iterations,
                random_state=42
            )
            coherence = CoherenceModel(
                model=model,
                texts=df_translated_documents['tokens'],
                dictionary=dictionary,
                coherence='c_v'
            ).get_coherence()
            perplexity = model.log_perplexity(heldout_corpus)
            results.append({
                'num_topics': num_topics,
                'passes' : passes,
                'iterations' : iterations,
                'alpha': model.alpha,
                'eta' : model.eta,
                'coherence': coherence,
                'perplexity': perplexity
            })
            print(results[-1])
            for i in range(model.num_topics):
                terms = [term for term, weight in model.show_topic(i, topn=20)]
                print(f"Topic {i+1}: {', '.join(terms)}")
            if coherence > best_coherence:
                best_coherence = coherence
                best_model = model
                best_params = {
                    'num_topics': num_topics,
                    'passes' : passes,
                    'iterations' : iterations,
                    'alpha': model.alpha,
                    'eta': model.eta,
                    'coherence': coherence,
                    'perplexity': perplexity
                }


In [ ]:
# 7. Display grid search results
#import ace_tools as tools
df_results = pd.DataFrame(results)
#tools.display_dataframe_to_user(name="LDA Grid Search Results", dataframe=df_results)

# 8. Show best model parameters and top topic terms
print("Best Model Parameters:", best_params)
for i in range(best_model.num_topics):
    terms = [term for term, weight in best_model.show_topic(i, topn=20)]
    print(f"Topic {i+1}: {', '.join(terms)}")

In [ ]:
df_results.to_excel('results.xlsx',index=False)
try:
    df_results.to_csv('results.csv',index=False)
except:
    pass

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# 1. Compute full document-topic distributions
doc_topic_probs = []
for bow in corpus:
    # Get probability for every topic in the model
    topic_dist = dict(best_model.get_document_topics(bow, minimum_probability=0))
    doc_topic_probs.append(topic_dist)

# 2. Convert to DataFrame
topic_df = pd.DataFrame(doc_topic_probs).fillna(0)
# Rename columns to topic_0, topic_1, ...
topic_df.columns = [f"topic_{i}" for i in topic_df.columns]

# 3. Combine with metadata
output_df = pd.concat(
    [df_documents[['country', 'filename']].reset_index(drop=True), topic_df],
    axis=1
)

output_df.to_excel("topic_dist_per_doc.xlsx", index=False)
output_df.to_csv("topic_dist_per_doc.csv", index=False)

In [ ]:
vis_data = gensimvis.prepare(best_model, corpus, dictionary)
pyLDAvis.enable_notebook()

vis_data